# Mouting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow

In [ ]:
# !pip install fasttext

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D, Input, Flatten, MaxPooling1D, SpatialDropout1D, Activation

from keras.callbacks import EarlyStopping

from numpy import array
from sklearn.metrics import classification_report

import gensim
from gensim import models
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

# Loading dataset

In [5]:
train_dataset = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/task datasets/original/train.csv")
val_dataset = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/task datasets/original/dev.csv")
test_dataset = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/task datasets/original/test.csv")

In [6]:
print(f'train: {train_dataset.shape}\nval: {val_dataset.shape}\ntest: {test_dataset.shape}')

train: (2700, 2)
val: (1330, 2)
test: (2016, 2)


In [7]:
train_dataset['label'].value_counts()

0    1389
1     922
2     389
Name: label, dtype: int64

In [8]:
train_x = train_dataset['text']
train_y = train_dataset['label']

val_x = val_dataset['text']
val_y = val_dataset['label']

test_x = test_dataset['text']
test_y = test_dataset['label']

# Embedding Setup

In [9]:
tokenizer=Tokenizer(oov_token = "<OOV>", split=' ') # Splitting text based on whitespace and adding "Out of vocabulary"
tokenizer.fit_on_texts(train_x) # Using the tokenizer on out train dataset to tokenize the train dataset
train_encoded=tokenizer.texts_to_sequences(train_x)
# print(train_encoded)

In [10]:
train_padded= pad_sequences(train_encoded, padding='post', maxlen=256)
# print(train_padded)

In [11]:
train_padded.shape[1]

256

In [12]:
# padding df_test
test_encoded=tokenizer.texts_to_sequences(test_x)
test_padded= pad_sequences(test_encoded, padding='post', maxlen=train_padded.shape[1])

In [13]:
# padding df_validation
val_encoded=tokenizer.texts_to_sequences(val_x)
val_padded= pad_sequences(val_encoded, padding='post', maxlen=train_padded.shape[1])

# Loading FastText

In [14]:
# import fasttext
import numpy as np

# # Load the FastText model
# fasttext_model = fasttext.load_model("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/notebooks/Tariq/fasttext/model_bn_300.bin")

# # Function that takes word vector as input and returns an embedding matrix
# def embedding_creation(EMBEDDING_DIM, word_vectors, tokenizer):
#     vocabulary_size = len(tokenizer.word_index) + 1
#     word_index = tokenizer.word_index
#     embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

#     for word, i in word_index.items():
#         try:
#             embedding_vector = word_vectors.get_word_vector(word)
#             embedding_matrix[i] = embedding_vector
#         except KeyError:
#             embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)

#     return embedding_matrix

# EMBEDDING_DIM = 300

# # Replace 'tokenizer' with the tokenizer object you have previously created
# # Assuming 'tokenizer' is defined earlier in your code

# embedding_matrix = embedding_creation(EMBEDDING_DIM, fasttext_model, tokenizer)


# Model Configuration

In [ ]:
!pip install scikit-learn

In [15]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# # Initialize and train the SVM classifier
# svm_classifier = SVC(kernel='rbf', C=1.0, decision_function_shape='ovr', random_state=42)
# svm_classifier.fit(train_padded, train_y)  # Assuming 'train_y' contains the labels

# # Make predictions on the validation set
# val_predictions = svm_classifier.predict(val_padded)

# # Evaluate the model on the validation set
# val_accuracy = accuracy_score(val_y, val_predictions)
# val_classification_report = classification_report(val_y, val_predictions)

# print(f"Validation Accuracy: {val_accuracy}")
# print("Validation Classification Report:\n", val_classification_report)

# # Make predictions on the test set
# test_predictions = svm_classifier.predict(test_padded)

# # Evaluate the model on the test set
# test_accuracy = accuracy_score(test_y, test_predictions)
# test_classification_report = classification_report(test_y, test_predictions)

# print(f"Test Accuracy: {test_accuracy}")
# print("Test Classification Report:\n", test_classification_report)


In [ ]:
import numpy as np
from gensim.models import Word2Vec
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the pre-trained CBOW Word2Vec model
cbow_model = Word2Vec.load("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/notebooks/Tariq/final/nafi/cbow_model")

# Assuming you have datasets with text and labels: train_dataset, val_dataset, test_dataset
# Replace 'texts' and 'labels' with your data
train_texts = train_dataset['text'].tolist()
val_texts = val_dataset['text'].tolist()
test_texts = test_dataset['text'].tolist()

train_labels = train_dataset['label'].tolist()
val_labels = val_dataset['label'].tolist()
test_labels = test_dataset['label'].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts + val_texts + test_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Pad the sequences to a fixed length
max_len = 256  # Set your desired maximum sequence length
train_sequences = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
val_sequences = pad_sequences(val_sequences, maxlen=max_len, padding='post', truncating='post')
test_sequences = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# Create arrays to hold the word embeddings for each sequence
embedding_dim = cbow_model.vector_size

def get_word_embeddings(sequences):
    word_embeddings = []
    for seq in sequences:
        seq_embeddings = []
        for token_idx in seq:
            word = tokenizer.index_word.get(token_idx, '')
            if word in cbow_model.wv:
                word_vector = cbow_model.wv[word]
            else:
                # Handle out-of-vocabulary words
                word_vector = np.zeros(embedding_dim)  # Replace with appropriate handling
            seq_embeddings.append(word_vector)
        word_embeddings.append(seq_embeddings)
    return np.array(word_embeddings)

# Convert sequences to word embeddings
X_train = get_word_embeddings(train_sequences)
X_val = get_word_embeddings(val_sequences)
X_test = get_word_embeddings(test_sequences)

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

# Initialize and train the SVM classifier on the training data
svm_classifier = svm.SVC(kernel='rbf')
svm_classifier.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# Make predictions on the validation set
val_predictions = svm_classifier.predict(X_val.reshape(X_val.shape[0], -1))

# Evaluate the model on the validation set
val_accuracy = accuracy_score(y_val, val_predictions)
val_classification_report = classification_report(y_val, val_predictions)

print(f"Validation Accuracy: {val_accuracy}")
print("Validation Classification Report:\n", val_classification_report)

# Make predictions on the test set
test_predictions = svm_classifier.predict(X_test.reshape(X_test.shape[0], -1))

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
test_classification_report = classification_report(y_test, test_predictions)

print(f"Test Accuracy: {test_accuracy}")
print("Test Classification Report:\n", test_classification_report)


In [ ]:
!pip install joblib

In [ ]:
import joblib
import os

# Save the trained model
model_filename = 'no_embedding_svm_model.pkl'
# Define the directory where you want to save the model
save_directory ="/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/notebooks/Tariq/final/nafi/"
# Combine the directory and filename to create the full file path
model_path = os.path.join(save_directory, model_filename)

joblib.dump(svm_classifier, model_path)

print(f"Model saved as {model_path}")

Model saved as /content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/notebooks/Tariq/final/nafi/no_embedding_svm_model.pkl


# Loading and testing

In [17]:
import joblib
import numpy as np
import os
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the directory and filename where the model is saved
model_directory = "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/notebooks/Tariq/final/nafi/"
model_filename = 'no_embedding_svm_model.pkl'

# Load the trained logistic regression model
model_path = os.path.join(model_directory, model_filename)
svm_classifier2 = joblib.load(model_path)

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

test_predictions = svm_classifier2.predict(X_test.reshape(X_test.shape[0], -1))


# Evaluate the model on the test set
test_accuracy = accuracy_score(test_y, test_predictions)
test_classification_report = classification_report(test_y, test_predictions, target_names=['0', '1', '2'], output_dict=True)

# Convert the classification report dictionary to a DataFrame
report_df = pd.DataFrame.from_dict(test_classification_report).T

# Format the DataFrame to display percentages
report_df['support'] = report_df['support'].astype(int)
report_df['accuracy'] = test_accuracy
report_df['accuracy'] = report_df['accuracy'].apply(lambda x: f'{x * 100:.2f}%')
report_df.iloc[:-1, :-1] = (report_df.iloc[:-1, :-1] * 100).applymap(lambda x: f'{x:.2f}%')

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print("Test Classification Report:\n", report_df)


Test Accuracy: 53.97%
Test Classification Report:
              precision    recall f1-score     support accuracy
0               54.51%    96.99%   69.80%  109600.00%   53.97%
1               40.35%     3.20%    5.93%   71900.00%   53.97%
2               22.22%     1.00%    1.90%   20100.00%   53.97%
accuracy        53.97%    53.97%   53.97%       0.00%   53.97%
macro avg       39.03%    33.73%   25.88%  201600.00%   53.97%
weighted avg  0.462426  0.539683  0.40249        2016   53.97%
